In [1]:
import numpy as np
import math
import matplotlib.pyplot as plt
import random

from braket.circuits import Circuit, circuit
from braket.devices import LocalSimulator
from braket.aws import AwsDevice
from fractions import Fraction
from math import gcd

import utils_qft
from utils_qpe import qpe, run_qpe

%load_ext autoreload
%autoreload 2
%matplotlib inline
%config IPCompleter.greedy=True

In [2]:
# 15に対するユニタリ作用素を作る。
# この作用素の位数 r を QPEで推定し、Shorのアルゴリズムで素因数を特定する。

def c_amod15(a):
    if a not in [2,7,8,11,13]:
        return None
    U = Circuit()        
    if a in [2,13]:
        U.swap(0,1)
        U.swap(1,2)
        U.swap(2,3)
    if a in [7,8]:
        U.swap(2,3)
        U.swap(1,2)
        U.swap(0,1)
    if a == 11:
        U.swap(1,3)
        U.swap(0,2)
    if a in [7,11,13]:
        for q in range(4):
            U.x(q)
    return U

In [3]:
# 基底 a とユニタリ作用素 unitary でのQPE

def qpe_amod(device, a, unitary, number_precision_qubits = 4, dim = 4):
    precision_qubits = range(number_precision_qubits)
    query_qubits = [i for i in range(number_precision_qubits, number_precision_qubits+dim)]
    query = Circuit().x(query_qubits[dim-1])

    # run qpe
    out = run_qpe(unitary, precision_qubits, query_qubits, query, device)

    # Get estimates phase.
    return out['phases_decimal']



In [4]:
# 推定した位相から位数を推定する

def phase2order(phase):
    frac = Fraction(phase[0]).limit_denominator(15)
    return frac.denominator

In [5]:
def shor(n, r):
    if r % 2 == 1:
        return None
    guesses = [gcd(a**(r//2)-1, 15), gcd(a**(r//2)+1, 15)]
    for guess in guesses:
        if guess != 1 and guess != 15 and (15 % guess) == 0:
            return guess
    return None


In [6]:
N = 15
unitary_generator = c_amod15
device = LocalSimulator()

attempt = 0
found = False
while not found:
    attempt += 1
    a = random.randint(2, N)
    print("\nAttempt %i: a=%i" % (attempt, a))

    unitary = unitary_generator(a)
    if (unitary == None):
        print("Failed. (illegal unitary operator)")
        continue

    phase = qpe_amod(device, a, unitary.as_unitary())
    if (phase == 0):
        print("Failed. (phase == 0)")
        continue
    r = phase2order(phase)
    print("Result: r = %i" % r)
    
    ret = shor(N, r)
    if (ret != None):
        print("*** Non-trivial factor found: %i ***" % ret)
        found = True
    else:
        print("Failed. (shor)")



Attempt 1: a=14
Failed. (illegal unitary operator)

Attempt 2: a=11
Result: r = 2
*** Non-trivial factor found: 5 ***
